# Personalized Recommendation system

- We will make personalized recomeendation system based on set of categories

- there are three categories created in this notebook :

**the first one is** : "Children" and it's criteria is : age less than 12 years

**the second one is** : "Teens"  and it's  criteria is:  age between 12 and 18 years

**the third one is** : "Adults" and it's criteria is :  age larger than 18 years

# About dataset

We use MovieLens 1M movie ratings. Stable benchmark dataset. 1 million ratings from 6000 users on 4000 movies. Released 2/2003.

these version of dataset contains "age - gender" Attributes for making recommendation for different clients based on there age

# Preparing Python Environment

In [1]:
#install needed libraries
!pip install scikit-surprise
!pip install scikit-surprise
!pip install --upgrade streamlit

# Deploy Model

In [2]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
import streamlit as st
import surprise
import numpy as np
import streamlit as st
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import surprise
from surprise import SVD
from surprise import accuracy
from surprise import Dataset, Reader, KNNBasic



# Load and merge the three tables
movies_data = pd.read_csv('movies.dat', names=["MovieID", "Title", "Genres"], sep="::")
ratings_data = pd.read_csv('ratings.dat', names=["UserID", "MovieID", "Rating", "Timestamp"], sep="::")
users_data = pd.read_csv("users.dat", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], sep="::")

# Merge tables based on common columns (e.g., MovieID and UserID)
merged_data = pd.merge(ratings_data, users_data, on='UserID')
merged_data = pd.merge(merged_data, movies_data, on='MovieID')

# Define groups and criteria
group_criteria = {
    'Children': (merged_data['Age'] <= 12),
    'Teens': (merged_data['Age'] > 12) & (merged_data['Age'] <= 18),
    'Adults': (merged_data['Age'] > 18),
}

# Save merged_data to a file (optional)
merged_data.to_csv('merged_data.csv', index=False)

# Build a collaborative filtering model (KNN) using the merged dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['UserID', 'MovieID', 'Rating']], reader)
trainset = data.build_full_trainset()
model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
model.fit(trainset)

# Save the trained model to a file
surprise.dump.dump('knn_model.pkl', algo=model)
# Save the merged data to a CSV file
merged_data.to_csv('merged_data.csv', index=False)

C:\Users\mostafa\AppData\Local\Temp\ipykernel_17684\2601738972.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_data = pd.read_csv('movies.dat', names=["MovieID", "Title", "Genres"], sep="::")
C:\Users\mostafa\AppData\Local\Temp\ipykernel_17684\2601738972.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_data = pd.read_csv('ratings.dat', names=["UserID", "MovieID", "Rating", "Timestamp"], sep="::")
C:\Users\mostafa\AppData\Local\Temp\ipykernel_17684\2601738972.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separat

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [3]:
%%writefile app3000.py
import streamlit as st
import surprise
import numpy as np
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import surprise
from surprise import SVD
from surprise import accuracy
from surprise import Dataset, Reader, KNNBasic


# Load the trained model from the file
model = surprise.dump.load('knn_model.pkl')[1]
merged_data = pd.read_csv('merged_data.csv')
movies_data = pd.read_csv('movies.dat', names=["MovieID", "Title", "Genres"], sep="::")
users_data = pd.read_csv("users.dat", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], sep="::")
group_criteria = {
    'Children': (merged_data['Age'] <= 12),
    'Teens': (merged_data['Age'] > 12) & (merged_data['Age'] <= 18),
    'Adults': (merged_data['Age'] > 18),
}

# Create a dictionary to map MovieID to Title
movie_id_to_title = dict(zip(movies_data['MovieID'], movies_data['Title']))



# Streamlit app
st.title("Movie Recommendations System")

# Input widgets
group_input = st.selectbox("Select a group:", list(group_criteria.keys()))
user_input = st.number_input("Enter a user ID:", min_value=1, max_value=users_data['UserID'].max())



# Function to get recommendations
def get_recommendations(group, user_id, top_n=10):
    if group not in group_criteria:
        return "Invalid group"

    user_in_group = merged_data[(merged_data['UserID'] == user_id) & group_criteria[group]].shape[0] > 0

    if not user_in_group:
        return "User not found in the specified group"

    user_history = merged_data[(merged_data['UserID'] == user_id) & (merged_data['Rating'] >= 4)]
    user_movie_ids = user_history['MovieID'].tolist()

    top_movie_ids = []

    if group == 'Children':
        genres_to_recommend = ['Children','Animation']
    elif group == 'Teens':
        genres_to_recommend = ['Drama', 'Action']
    elif group == 'Adults':
        genres_to_recommend = ['Romance','Comedy']
    else:
        genres_to_recommend = []

    for movie_id in range(1, 3953):
        if movie_id not in user_movie_ids:
            movie_genres = movies_data.loc[movies_data['MovieID'] == movie_id, 'Genres']
            if not movie_genres.empty and any(genre in movie_genres.values[0] for genre in genres_to_recommend):
                prediction = model.predict(user_id, movie_id)
                top_movie_ids.append((movie_id, prediction.est))

    top_movie_ids.sort(key=lambda x: x[1], reverse=True)
    top_movie_ids = [movie[0] for movie in top_movie_ids[:top_n]]

    top_movie_titles = [movie_id_to_title[movie_id].rsplit('(', 1)[0].strip() for movie_id in top_movie_ids]

    return top_movie_titles

# Button to get recommendations
if st.button("Get Recommendations"):
    recommendations = get_recommendations(group_input, user_input)
    if isinstance(recommendations, list):
        st.write(f"Top {len(recommendations)} Movie Recommendations for {group_input} Group and User {user_input}:")
        for movie_title in recommendations:
            st.write(movie_title)
    else:
        st.write(recommendations)

Overwriting app3000.py


In [ ]:
!streamlit run app3000.py